# Custom Challenge #001: Maze Traversal

## SUMMARY
Traverse a maze from start to finish.

## DETAILS
A maze is represented by a 2D array of strings and may contain the following values:
- 'S': Start
- 'E': Finish
- 'X': Wall/Obstacle
- ' ': Empty Space

A move is a traversal by one row or one column in the following directions:
- Up, Right, Down, or Left

EXAMPLE #1
```
Input:
    X X X X X X X
    X   X X   S X
    X           X
    X       X   X
    X X X   X X X
    X         E X
    X X X X X X X
```

Output
```
[(1, 5), (2, 5), (2, 4), (2, 3), (3, 3), (4, 3), (5, 3), (5, 4), (5, 5)]
or
[(1, 5), (1, 4), (2, 4), (2, 3), (3, 3), (4, 3), (5, 3), (5, 4), (5, 5)]
```

Explanation:
```
The solution can be seen below, where the dots (•) represent the shortest path from start to finish:
      0 1 2 3 4 5 6
    0 X X X X X X X
    1 X   X X   S X
    2 X     • • • X
    3 X     • X   X
    4 X X X • X X X
    5 X     • • • X
    6 X X X X X X X
```

In [9]:
from collections import deque
import copy

In [10]:
class MazeSolver:
    """Maze Solver"""
    def __init__(self):
        self._n_cols = None
        self._n_rows = None
        self._maze = None
        self._start_pos = None, None
        self._visited = None
        self._solution = None
        self._bfs_queue = deque()
        self._directions = [
            [-1, 0], # Up
            [0, 1],  # Right
            [1, 0],  # Down
            [0, -1]  # Left
        ]
        
    # Initialization methods

    def load_maze(self, maze):
        if not maze or type(maze) != list or not maze[0] or maze[0] == []:
            raise Exception('Attempted to load invalid maze.')

        self.__init__()
        self._maze = maze
        self._n_rows = len(self._maze)
        self._n_cols = len(self._maze[0])
        self._start_pos = self._get_start_pos()
        self._visited = [
            [False for _ in range(self._n_cols)]
            for _ in range(self._n_rows)
        ]

    def _get_start_pos(self) -> tuple:
        for row in range(self._n_rows):
            for col in range(self._n_cols):
                if self._maze[row][col] == 'S':
                    return row, col
        return None, None
                
    # Utility
                
    def _enqueue(self, coord):
        self._bfs_queue.append(coord)
    
    def _dequeue(self):
        return self._bfs_queue.popleft()
    
    def _is_valid_coord(self, coord: tuple) -> bool:
        row, col = coord
        
        # Check if coordinate is within bounds of the array
        if row < 0 or row >= self._n_rows or col < 0 or col >= self._n_cols:
            return False
          
        # Check if coordinate contains an obstacle
        if self._maze[row][col] == 'X':
            return False

        # Check if coordinate has been visited before
        if self._visited[row][col]:
            return False
        
        return True
    
    def draw_solution(self):
        path = self._solution
        path.pop(0)
        path.pop()
        
        maze = self._maze.copy()
        
        for row, col in path:
            maze[row][col] = '•'
        
        for row in maze:
            for val in row:
                print(val, end=' ')
            print()
    
    # Navigation

    def _dfs(self, curr=None, path=None, visited=None) -> bool:
        if curr is None:
            curr = self._start_pos
        if path is None:
            path = []
        if visited is None:
            visited = self._visited

        row, col = curr
        curr_val = self._maze[row][col]

        if not self._is_valid_coord(curr):
            return False

        visited[row][col] = True
        path.append(curr)

        if curr_val == 'E':
            self._solution = path.copy()
            return True

        for _y, _x in self._directions:
            next_coord = row + _y, col + _x
            if self._dfs(next_coord, path, visited):
                return True

        # Undo the modifications after the recursive call
        visited[row][col] = False
        path.pop()

        return False

    def _solve_dfs(self):
        self._dfs()
        return copy.deepcopy(self._solution)

    def _solve_bfs(self):
        self._enqueue([self._start_pos])
        path = []

        while len(self._bfs_queue):
            path = self._dequeue()
            row, col = path[-1]
            curr_val = self._maze[row][col]

            if curr_val == 'E':
                self._solution = path
                return path

            for _y, _x in self._directions:
                next_coord = row + _y, col + _x
                next_row, next_col = next_coord

                if not self._is_valid_coord(next_coord):
                    continue

                self._enqueue(path + [next_coord])
                self._visited[next_row][next_col] = True

        self._solution = path

        return copy.deepcopy(self._solution)
        
    def solve_maze(self, algorithm='BFS'):
        if self._solution:
            self.load_maze(self._maze)

        return self._solve_bfs() if algorithm == 'BFS' else self._solve_dfs()  # use ternary expression

Define test mazes and execute a test suite

In [11]:
test_mazes = [
    {
        'name': 'Two simple paths',
        'maze': [
            ['X', 'X', 'X', 'X', 'X', 'X', 'X'],
            ['X', ' ', 'X', 'X', ' ', 'S', 'X'],
            ['X', ' ', ' ', ' ', ' ', ' ', 'X'],
            ['X', ' ', ' ', ' ', 'X', ' ', 'X'],
            ['X', 'X', 'X', ' ', 'X', 'X', 'X'],
            ['X', ' ', ' ', ' ', ' ', 'E', 'X'],
            ['X', 'X', 'X', 'X', 'X', 'X', 'X']
        ],
        'start_pos': (1, 5),
        'end_pos': (5, 5),
        'solutions': [
            [(1, 5), (2, 5), (2, 4), (2, 3), (3, 3), (4, 3), (5, 3), (5, 4), (5, 5)]
        ]
    },
    {
        'name': 'Many choices',
        'maze': [
            ['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'],
            ['X', ' ', 'X', 'X', ' ', ' ', ' ', ' ', 'X', 'X'],
            ['X', ' ', ' ', ' ', ' ', ' ', 'X', ' ', 'X', 'X'],
            ['X', ' ', ' ', ' ', 'X', 'S', 'X', ' ', ' ', 'X'],
            ['X', 'X', 'X', ' ', 'X', 'X', 'X', 'X', ' ', 'X'],
            ['X', 'X', 'E', ' ', ' ', ' ', ' ', ' ', ' ', 'X'],
            ['X', 'X', ' ', 'X', ' ', 'X', 'X', 'X', ' ', 'X'],
            ['X', 'X', ' ', 'X', ' ', ' ', ' ', 'X', ' ', 'X'],
            ['X', ' ', ' ', ' ', ' ', 'X', ' ', ' ', ' ', 'X'],
            ['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
        ],
        'start_pos': (3, 5),
        'end_pos': (5, 2),
        'solutions': [
            [(3, 5), (2, 5), (1, 5), (1, 6), (1, 7), (2, 7), (3, 7), (3, 8), (4, 8), (5, 8), (6, 8), (7, 8), (8, 8), (8, 7), (8, 6), (7, 6), (7, 5), (7, 4), (6, 4), (5, 4), (5, 3), (5, 2)],
            [(3, 5), (2, 5), (2, 4), (2, 3), (3, 3), (4, 3), (5, 3), (5, 2)]
        ]
    }
]

In [12]:

solver = MazeSolver()

for i, test in enumerate(test_mazes):
    solver.load_maze(test["maze"])
    solution_bfs = solver.solve_maze(algorithm='BFS')
    solution_dfs = solver.solve_maze(algorithm='DFS')
    passed = solution_bfs in test['solutions'] and solution_dfs in test['solutions']
    result = 'PASS' if passed else 'FAIL'
    print(f'{i + 1}. {test["name"]} – {result}')

1. Two simple paths – PASS
2. Many choices – PASS
